In [1]:
import sys
import numpy as np
from keras.utils import np_utils
from keras.models import Sequential
from keras.callbacks import ModelCheckpoint
from keras.layers import Dense, Dropout, LSTM
from keras.layers.embeddings import Embedding

Using Theano backend.
Using gpu device 0: GeForce GT 755M (CNMeM is enabled with initial size: 80.0% of memory, cuDNN not available)


In [2]:
raw_text = open('wonderland.txt').read()
raw_text = raw_text.lower()
print raw_text[:200]

alice's adventures in wonderland

lewis carroll

the millennium fulcrum edition 3.0

chapter i. down the rabbit-hole

alice was beginning to get very tired of sitting by her sister on the
bank, and of


In [3]:
chars = sorted(list(set(raw_text)))
n_chars = len(raw_text)
n_vocab = len(chars)
print "Total Characters:", n_chars
print "Total Vocab:", n_vocab
char_to_int = dict((c, i) for i, c in enumerate(chars))
int_to_char = dict((i, c) for i, c in enumerate(chars))

Total Characters: 144413
Total Vocab: 45


In [4]:
from nltk import tokenize
sentences = tokenize.sent_tokenize(raw_text)
lengths = []
for i in xrange(len(sentences)):
    lengths.append(len(sentences[i]))
lengths = np.array(lengths)
sentences[0]

"alice's adventures in wonderland\n\nlewis carroll\n\nthe millennium fulcrum edition 3.0\n\nchapter i. down the rabbit-hole\n\nalice was beginning to get very tired of sitting by her sister on the\nbank, and of having nothing to do: once or twice she had peeped into the\nbook her sister was reading, but it had no pictures or conversations in\nit, 'and what is the use of a book,' thought alice 'without pictures or\nconversations?'"

In [5]:
seq_length = 100
dataX = []
dataY = []
for i in xrange(0, n_chars - seq_length):
    seq_in  = raw_text[i: i+seq_length]
    seq_out = raw_text[i+seq_length]
    dataX.append([char_to_int[char] for char in seq_in])
    dataY.append(char_to_int[seq_out])
n_patterns = len(dataX)
print 'Total patterns:', n_patterns

Total patterns: 144313


# Small LSTM Network : Constant Characters

In [6]:
# Reshape dataX to size of [samples, time steps, features] and scale it to 0-1
# Represent dataY as one hot encoding
X_train = np.reshape(dataX, (n_patterns, seq_length, 1))/float(n_vocab)
Y_train = np_utils.to_categorical(dataY)

In [7]:
model = Sequential()
model.add(LSTM(256, input_shape=(X_train.shape[1], X_train.shape[2])))
model.add(Dropout(0.2))
model.add(Dense(Y_train.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')
print model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
lstm_1 (LSTM)                    (None, 256)           264192      lstm_input_1[0][0]               
____________________________________________________________________________________________________
dropout_1 (Dropout)              (None, 256)           0           lstm_1[0][0]                     
____________________________________________________________________________________________________
dense_1 (Dense)                  (None, 45L)           11565       dropout_1[0][0]                  
Total params: 275757
____________________________________________________________________________________________________
None


In [8]:
# define the checkpoint
filepath="weights-improvement-{epoch:02d}-{loss:.4f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]

In [9]:
model.fit(X_train, Y_train, nb_epoch=20, batch_size=64, callbacks=callbacks_list)

Epoch 1/20
144313/144313 [==============================] - 929s - loss: 2.8973   
Epoch 2/20
144313/144313 [==============================] - 918s - loss: 2.6795   
Epoch 3/20
144313/144313 [==============================] - 913s - loss: 2.5778   
Epoch 4/20
144313/144313 [==============================] - 905s - loss: 2.5085   
Epoch 5/20
144313/144313 [==============================] - 903s - loss: 2.4442   
Epoch 6/20
144313/144313 [==============================] - 905s - loss: 2.3870   
Epoch 7/20
144313/144313 [==============================] - 902s - loss: 2.3312   
Epoch 8/20
144313/144313 [==============================] - 902s - loss: 2.2781   
Epoch 9/20
144313/144313 [==============================] - 899s - loss: 2.2292   
Epoch 10/20
144313/144313 [==============================] - 898s - loss: 2.1817   
Epoch 11/20
144313/144313 [==============================] - 898s - loss: 2.1408   
Epoch 12/20
144313/144313 [==============================] - 898s - loss: 2.1014   
E

In [11]:
# load the network weights
filename = "weights-improvement-16-2.0185.hdf5"
model.load_weights(filename)
model.compile(loss='categorical_crossentropy', optimizer='adam')

In [14]:
start = np.random.randint(0, len(X_train)-1)
pattern = dataX[start]
print "Seed:"
print "\"", ''.join([int_to_char[value] for value in pattern]), "\""
print "\nGenerated Sequence:"
for i in xrange(200):
    x = np.reshape(pattern, (1, len(pattern), 1))
    x = x/float(n_vocab)
    prediction = model.predict(x)
    index = np.argmax(prediction)
    result = int_to_char[index]
    sys.stdout.write(result)
    pattern.append(index)
    pattern = pattern[1:len(pattern)]
print "\nDone."    

Seed:
" e
to them, and all dripping wet, cross, and uncomfortable.

the first question of course was, how to "
 soenk at the could ao a lore of the sabbit sar and aalen and all the doere sf teered the had beee oo the daneer  and whet soeee to toink at ierse the saabit sareen the was aol a crea and an a lore of
Done.


# Small LSTM Network : Constant Characters With Embedding Layers

In [6]:
X_train = np.reshape(dataX, (n_patterns, seq_length))/float(n_vocab)
Y_train = np_utils.to_categorical(dataY)

In [7]:
embedding_vector_length = 32
model1 = Sequential()
model1.add(Embedding(n_vocab, embedding_vector_length, input_length=seq_length))
model1.add(LSTM(256))
model1.add(Dropout(0.2))
model1.add(Dense(Y_train.shape[1], activation='softmax'))
model1.compile(loss='categorical_crossentropy', optimizer='adam')
print model1.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
embedding_1 (Embedding)          (None, 100, 32)       1440        embedding_input_1[0][0]          
____________________________________________________________________________________________________
lstm_1 (LSTM)                    (None, 256)           295936      embedding_1[0][0]                
____________________________________________________________________________________________________
dropout_1 (Dropout)              (None, 256)           0           lstm_1[0][0]                     
____________________________________________________________________________________________________
dense_1 (Dense)                  (None, 45L)           11565       dropout_1[0][0]                  
Total params: 308941
______________________________________________________________________

In [8]:
model1.fit(X_train, Y_train, nb_epoch=10, batch_size=512)

Epoch 1/10
144313/144313 [==============================] - 343s - loss: 3.0683   
Epoch 2/10
144313/144313 [==============================] - 344s - loss: 3.0407   
Epoch 3/10
144313/144313 [==============================] - 344s - loss: 3.0370   
Epoch 4/10
144313/144313 [==============================] - 343s - loss: 3.0347   
Epoch 5/10
144313/144313 [==============================] - 341s - loss: 3.0336   
Epoch 6/10
144313/144313 [==============================] - 341s - loss: 3.0331   
Epoch 7/10
144313/144313 [==============================] - 339s - loss: 3.0320   
Epoch 8/10
144313/144313 [==============================] - 337s - loss: 3.0319   
Epoch 9/10
144313/144313 [==============================] - 506s - loss: 3.0321   
Epoch 10/10
144313/144313 [==============================] - 574s - loss: 3.0319   


In [ ]:
start = np.random.randint(0, len(X_train)-1)
pattern = dataX[start]
print "Seed:"
print "\"", ''.join([int_to_char[value] for value in pattern]), "\""
for i in xrange(500):
    x = np.reshape(pattern, (1, len(pattern)))
    x = x/float(n_vocab)
    prediction = model1.predict(x)
    index = np.argmax(prediction)
    result = int_to_char[index]
    sys.stdout.write(result)
    pattern.append(index)
    pattern = pattern[1:len(pattern)]
print "\nDone."    

# Deep LSTM Network : Constant Characters

In [32]:
# Reshape dataX to size of [samples, time steps, features] and scale it to 0-1
# Represent dataY as one hot encoding
X_train = np.reshape(dataX, (n_patterns, seq_length, 1))/float(n_vocab)
Y_train = np_utils.to_categorical(dataY)

In [51]:
model = Sequential()
model.add(LSTM(256, input_shape=(X_train.shape[1], X_train.shape[2]), return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(64))
model.add(Dropout(0.2))
model.add(Dense(Y_train.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')
print model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
lstm_14 (LSTM)                   (None, 100L, 256)     264192      lstm_input_8[0][0]               
____________________________________________________________________________________________________
dropout_11 (Dropout)             (None, 100L, 256)     0           lstm_14[0][0]                    
____________________________________________________________________________________________________
lstm_15 (LSTM)                   (None, 64)            82176       dropout_11[0][0]                 
____________________________________________________________________________________________________
dropout_12 (Dropout)             (None, 64)            0           lstm_15[0][0]                    
___________________________________________________________________________________________

In [53]:
# define the checkpoint
filepath="weights-improvement-{epoch:02d}-{loss:.4f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]

In [2]:
model.fit(X_train, Y_train, nb_epoch=2, batch_size=256, callbacks=callbacks_list)